In [174]:
modelCols=["credit_level","gender","inet_pd_inst_cnt","star_level",
    "dt_m_1000","dt_m_1003","dt_m_1004","dt_m_1005","dt_m_1006","dt_m_1009","dt_m_1011","dt_m_1012","dt_m_1015","dt_m_1017",
    "dt_m_1027","dt_m_1028","dt_m_1041","dt_m_1044","dt_m_1051","dt_m_1053","dt_m_1067",
    "dt_m_1069","dt_m_1073","dt_m_1074","dt_m_1075","dt_m_1085","dt_m_1086","dt_m_1087","dt_m_1096","dt_m_1099","dt_m_1102",
    "dt_m_1105","dt_m_1108","dt_m_1111","dt_m_1594","dt_m_1601","dt_m_1617","dt_m_1618","dt_m_1620","dt_m_1630","dt_m_1633",
    "last_year_capture_user_flag","app1_visits","app2_visits","app3_visits","app4_visits","app5_visits","app6_visits",
    "app7_visits","app8_visits","access_net_dur","tmlRegister_net_dur","prdOpen_dur","tmlRegister_dur","prdOpen_net_dur",
    "brand","product","cust_point_level","age","market_price_level","app_cnt",
    "dt_m_1027_type","dt_m_1075_type","dt_m_1086_type","dt_m_1087_type",
    "dt_m_1102_type","dt_m_1108_type","dt_m_1594_type","dt_m_1617_type",
    "dt_m_1630_type","dt_m_1633_type","app1_visits_type","app2_visits_type","app3_visits_type",
    "app4_visits_type","app5_visits_type","app6_visits_type","app7_visits_type",
    "dt_m_1087_pref","in_10s_per","in_10_30s_per","in_30_60s_per","out_60s_per","dt_m_1035_pref","dt_m_1043","dt_m_1052",
    "zjdur_per_times","bjdur_per_times","zjdur_perc","bjdur_perc","zjdur_per_man","zjman_perc","zjman_per_recent3","dt_m_1085_bj",
    "dt_m_1086_bj","dt_m_1087_bj","dt_m_1099_perc","dt_m_1105_perc","zjctdur_per_times","bjctdur_per_times","market_price_per_mon",
    "ctdur_per_times","zjctdur_perc","total_message","message_send_perc","zjtimes_per_day","bjtimes_per_day","zjdur_mean_vs_max",
    "bjdur_mean_vs_max","current_month_man_perc","zj_current_month_man_perc","dt_m_1034","dt_m_1035",'liuliang_per_dur',
    'app_pref','netdur_per_appvisit','netflow_per_appvisit','flowmean_per_prod','zjdurmean_per_prod','zjdur_per_prodcnt',
    'bjdur_per_prodcnt','flow_per_prodcnt','message_per_prodcnt','rest_zj_perc','zjman_per_day','zjtime_per_man','bjtimes_per_man','bjdur_per_man',
    'zjdur_per_man_perc','zjtimes_per_man_perc','bj_man','bj_man_perc','zj_bj_man','bjman_per_day']

In [134]:
len(modelCols)

133

In [135]:
catCols=["credit_level","gender","inet_pd_inst_cnt","star_level", "last_year_capture_user_flag","brand","product","cust_point_level","market_price_level","app_cnt",
    "dt_m_1027_type","dt_m_1075_type","dt_m_1086_type","dt_m_1087_type",
    "dt_m_1102_type","dt_m_1108_type","dt_m_1594_type","dt_m_1617_type",
    "dt_m_1630_type","dt_m_1633_type","app1_visits_type","app2_visits_type","app3_visits_type",
    "app4_visits_type","app5_visits_type","app6_visits_type","app7_visits_type",'app_pref']

In [136]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics.classification import f1_score
from xgboost import plot_importance
from matplotlib import pyplot

In [137]:
data=pd.read_csv("E:\zte\data\data_v5.csv")
data['bjtimes_per_man']=data['calltime_per_man']-data['zjtime_per_man']
data['bjdur_per_man']=data['calldur_per_man']-data['zjdur_per_man']
data['zjdur_per_man_perc']=data['zjdur_per_man']/data['calldur_per_man']
data['zjtimes_per_man_perc']=data['zjtime_per_man']/data['calltime_per_man']
data['bj_man']=data['dt_m_1015']/data['bjdur_per_man'].replace(0, np.inf) 
data['bj_man_perc']=data['bj_man']/data['dt_m_1041'].replace(0, np.inf) 
data['zj_bj_man']=data['bj_man']+data['dt_m_1051']-data['dt_m_1041']
data['bjman_per_day']=data['bj_man']/(31-data['dt_m_1028']).replace(0, np.inf)

In [138]:
lb=LabelEncoder()
data['app_pref']=data['app_pref'].fillna("app0_visits")
for col in ["brand","product",'app_pref']:
    data[col]=lb.fit_transform(data[col])

In [139]:
submitData=data[data["label"].isnull()]
submitData["user"].count()

79407

In [140]:
trainData=data[~data["label"].isnull()]
trainData["user"].count()

322757

In [175]:
trainData["label_index"]=lb.fit_transform(trainData["label"])
# X=trainData[modelCols]
y=trainData["label_index"]

D:\Users\fjc-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [176]:
train_x_all, test_x_all, train_y_all, test_y=train_test_split(trainData,y,random_state=88888)
m=train_x_all[train_x_all["label"]=='7C26FADD409BD4B9']
n=train_x_all[train_x_all["label"]!='7C26FADD409BD4B9']
sample=m.sample(50000,random_state=963741)
train=n.append(sample)
train_x=train[modelCols]
train_y=train["label_index"]
tmp_train=train_x[modelCols]
train_data=lgb.Dataset(tmp_train,train_y,categorical_feature=catCols,free_raw_data=False)
tmp=test_x_all[modelCols]
test_data=lgb.Dataset(tmp,test_y,reference=train_data,free_raw_data=False)

In [177]:
params = {
    'boosting_type': 'goss',
    'objective': 'multiclass',
    'num_class':5,
    'metric': 'multi_logloss',
    'n_estimators':2000,
    'max_depth': 8,
    'num_leaves':30,
    'learning_rate': 0.04,
    'feature_fraction': 0.1,
    'feature_fraction_bynode':0.7,
    'top_rate':0.3,
    'other_rate':0.1,
    'lambda_l1':0,
    'lambda_l2':1,
    'num_threads':10
}

In [178]:
gbm=lgb.train(params,train_data,valid_sets={train_data,test_data},early_stopping_rounds=10)

D:\Users\fjc-pc\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
D:\Users\fjc-pc\Anaconda3\lib\site-packages\lightgbm\basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
D:\Users\fjc-pc\Anaconda3\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['app1_visits_type', 'app2_visits_type', 'app3_visits_type', 'app4_visits_type', 'app5_visits_type', 'app6_visits_type', 'app7_visits_type', 'app_cnt', 'app_pref', 'brand', 'credit_level', 'cust_point_level', 'dt_m_1027_type', 'dt_m_1075_type', 'dt_m_1086_type', 'dt_m_1087_type', 'dt_m_1102_type', 'dt_m_1108_type', 'dt_m_1594_type', 'dt_m_1617_type', 'dt_m_1630_type', 'dt_m_1633_type', 'gender', 'inet_pd_inst_cnt', 'last_year_capture_

[1]	training's multi_logloss: 1.06024	valid_1's multi_logloss: 0.654027
Training until validation scores don't improve for 10 rounds
[2]	training's multi_logloss: 1.01348	valid_1's multi_logloss: 0.629003
[3]	training's multi_logloss: 0.968031	valid_1's multi_logloss: 0.604913
[4]	training's multi_logloss: 0.930213	valid_1's multi_logloss: 0.583749
[5]	training's multi_logloss: 0.894763	valid_1's multi_logloss: 0.56334
[6]	training's multi_logloss: 0.860425	valid_1's multi_logloss: 0.543406
[7]	training's multi_logloss: 0.83122	valid_1's multi_logloss: 0.526128
[8]	training's multi_logloss: 0.804148	valid_1's multi_logloss: 0.509766
[9]	training's multi_logloss: 0.779776	valid_1's multi_logloss: 0.494947
[10]	training's multi_logloss: 0.757439	valid_1's multi_logloss: 0.481403
[11]	training's multi_logloss: 0.734436	valid_1's multi_logloss: 0.467249
[12]	training's multi_logloss: 0.712675	valid_1's multi_logloss: 0.453952
[13]	training's multi_logloss: 0.691866	valid_1's multi_logloss:

[113]	training's multi_logloss: 0.222136	valid_1's multi_logloss: 0.129344
[114]	training's multi_logloss: 0.221293	valid_1's multi_logloss: 0.128952
[115]	training's multi_logloss: 0.220546	valid_1's multi_logloss: 0.128599
[116]	training's multi_logloss: 0.219766	valid_1's multi_logloss: 0.128199
[117]	training's multi_logloss: 0.218791	valid_1's multi_logloss: 0.127743
[118]	training's multi_logloss: 0.218113	valid_1's multi_logloss: 0.127505
[119]	training's multi_logloss: 0.217204	valid_1's multi_logloss: 0.127059
[120]	training's multi_logloss: 0.216636	valid_1's multi_logloss: 0.126843
[121]	training's multi_logloss: 0.2159	valid_1's multi_logloss: 0.126517
[122]	training's multi_logloss: 0.215257	valid_1's multi_logloss: 0.126199
[123]	training's multi_logloss: 0.214651	valid_1's multi_logloss: 0.125912
[124]	training's multi_logloss: 0.2139	valid_1's multi_logloss: 0.125604
[125]	training's multi_logloss: 0.213184	valid_1's multi_logloss: 0.125257
[126]	training's multi_loglos

[225]	training's multi_logloss: 0.168517	valid_1's multi_logloss: 0.109994
[226]	training's multi_logloss: 0.168181	valid_1's multi_logloss: 0.109917
[227]	training's multi_logloss: 0.167848	valid_1's multi_logloss: 0.109812
[228]	training's multi_logloss: 0.167539	valid_1's multi_logloss: 0.109708
[229]	training's multi_logloss: 0.167265	valid_1's multi_logloss: 0.10963
[230]	training's multi_logloss: 0.166917	valid_1's multi_logloss: 0.109528
[231]	training's multi_logloss: 0.166614	valid_1's multi_logloss: 0.109471
[232]	training's multi_logloss: 0.16632	valid_1's multi_logloss: 0.109391
[233]	training's multi_logloss: 0.16604	valid_1's multi_logloss: 0.109333
[234]	training's multi_logloss: 0.165739	valid_1's multi_logloss: 0.109265
[235]	training's multi_logloss: 0.16543	valid_1's multi_logloss: 0.109192
[236]	training's multi_logloss: 0.165155	valid_1's multi_logloss: 0.109147
[237]	training's multi_logloss: 0.164887	valid_1's multi_logloss: 0.109077
[238]	training's multi_loglos

[338]	training's multi_logloss: 0.140843	valid_1's multi_logloss: 0.103689
[339]	training's multi_logloss: 0.140632	valid_1's multi_logloss: 0.103651
[340]	training's multi_logloss: 0.14043	valid_1's multi_logloss: 0.103608
[341]	training's multi_logloss: 0.140232	valid_1's multi_logloss: 0.103572
[342]	training's multi_logloss: 0.140031	valid_1's multi_logloss: 0.103536
[343]	training's multi_logloss: 0.139848	valid_1's multi_logloss: 0.103501
[344]	training's multi_logloss: 0.139661	valid_1's multi_logloss: 0.103486
[345]	training's multi_logloss: 0.139453	valid_1's multi_logloss: 0.103445
[346]	training's multi_logloss: 0.139263	valid_1's multi_logloss: 0.1034
[347]	training's multi_logloss: 0.139058	valid_1's multi_logloss: 0.103371
[348]	training's multi_logloss: 0.13889	valid_1's multi_logloss: 0.103341
[349]	training's multi_logloss: 0.138697	valid_1's multi_logloss: 0.103315
[350]	training's multi_logloss: 0.138479	valid_1's multi_logloss: 0.103247
[351]	training's multi_loglos

...

[777]	training's multi_logloss: 0.0827675	valid_1's multi_logloss: 0.0958797
[778]	training's multi_logloss: 0.0826734	valid_1's multi_logloss: 0.0958723
[779]	training's multi_logloss: 0.0825935	valid_1's multi_logloss: 0.0958732
[780]	training's multi_logloss: 0.0825112	valid_1's multi_logloss: 0.0958741
[781]	training's multi_logloss: 0.0824153	valid_1's multi_logloss: 0.095861
[782]	training's multi_logloss: 0.082322	valid_1's multi_logloss: 0.0958613
[783]	training's multi_logloss: 0.0822252	valid_1's multi_logloss: 0.0958429
[784]	training's multi_logloss: 0.0821447	valid_1's multi_logloss: 0.0958312
[785]	training's multi_logloss: 0.0820687	valid_1's multi_logloss: 0.0958292
[786]	training's multi_logloss: 0.0819838	valid_1's multi_logloss: 0.0958248
[787]	training's multi_logloss: 0.0818984	valid_1's multi_logloss: 0.0958139
[788]	training's multi_logloss: 0.0817903	valid_1's multi_logloss: 0.0958101
[789]	training's multi_logloss: 0.0816887	valid_1's multi_logloss: 0.0957906
[

[887]	training's multi_logloss: 0.073417	valid_1's multi_logloss: 0.0949777
[888]	training's multi_logloss: 0.0733363	valid_1's multi_logloss: 0.0949729
[889]	training's multi_logloss: 0.0732522	valid_1's multi_logloss: 0.0949607
[890]	training's multi_logloss: 0.0731866	valid_1's multi_logloss: 0.0949581
[891]	training's multi_logloss: 0.0731156	valid_1's multi_logloss: 0.0949629
[892]	training's multi_logloss: 0.0730344	valid_1's multi_logloss: 0.0949549
[893]	training's multi_logloss: 0.0729552	valid_1's multi_logloss: 0.0949429
[894]	training's multi_logloss: 0.0728842	valid_1's multi_logloss: 0.0949339
[895]	training's multi_logloss: 0.072809	valid_1's multi_logloss: 0.0949365
[896]	training's multi_logloss: 0.0727295	valid_1's multi_logloss: 0.0949317
[897]	training's multi_logloss: 0.0726655	valid_1's multi_logloss: 0.09494
[898]	training's multi_logloss: 0.0725874	valid_1's multi_logloss: 0.0949273
[899]	training's multi_logloss: 0.0725096	valid_1's multi_logloss: 0.0949101
[90

[994]	training's multi_logloss: 0.0657326	valid_1's multi_logloss: 0.0944367
[995]	training's multi_logloss: 0.0656746	valid_1's multi_logloss: 0.0944409
[996]	training's multi_logloss: 0.0656069	valid_1's multi_logloss: 0.0944298
[997]	training's multi_logloss: 0.0655443	valid_1's multi_logloss: 0.0944305
[998]	training's multi_logloss: 0.0654657	valid_1's multi_logloss: 0.0944272
[999]	training's multi_logloss: 0.0654068	valid_1's multi_logloss: 0.0944252
[1000]	training's multi_logloss: 0.0653417	valid_1's multi_logloss: 0.0944245
[1001]	training's multi_logloss: 0.0652736	valid_1's multi_logloss: 0.0944223
[1002]	training's multi_logloss: 0.0652075	valid_1's multi_logloss: 0.0944168
[1003]	training's multi_logloss: 0.0651395	valid_1's multi_logloss: 0.0944146
[1004]	training's multi_logloss: 0.0650739	valid_1's multi_logloss: 0.0944136
[1005]	training's multi_logloss: 0.0650165	valid_1's multi_logloss: 0.0944114
[1006]	training's multi_logloss: 0.064951	valid_1's multi_logloss: 0.0

[1100]	training's multi_logloss: 0.0590196	valid_1's multi_logloss: 0.0940778
[1101]	training's multi_logloss: 0.0589683	valid_1's multi_logloss: 0.0940865
[1102]	training's multi_logloss: 0.0589055	valid_1's multi_logloss: 0.0940814
[1103]	training's multi_logloss: 0.0588503	valid_1's multi_logloss: 0.0940789
[1104]	training's multi_logloss: 0.0587866	valid_1's multi_logloss: 0.0940709
[1105]	training's multi_logloss: 0.0587223	valid_1's multi_logloss: 0.0940644
[1106]	training's multi_logloss: 0.058656	valid_1's multi_logloss: 0.0940585
[1107]	training's multi_logloss: 0.0585959	valid_1's multi_logloss: 0.0940578
[1108]	training's multi_logloss: 0.058546	valid_1's multi_logloss: 0.0940565
[1109]	training's multi_logloss: 0.0584896	valid_1's multi_logloss: 0.0940514
[1110]	training's multi_logloss: 0.0584233	valid_1's multi_logloss: 0.0940477
[1111]	training's multi_logloss: 0.0583633	valid_1's multi_logloss: 0.094042
[1112]	training's multi_logloss: 0.0583133	valid_1's multi_logloss:

[1208]	training's multi_logloss: 0.0529853	valid_1's multi_logloss: 0.0937038
[1209]	training's multi_logloss: 0.05293	valid_1's multi_logloss: 0.0936947
[1210]	training's multi_logloss: 0.052875	valid_1's multi_logloss: 0.0936972
[1211]	training's multi_logloss: 0.0528167	valid_1's multi_logloss: 0.0936882
[1212]	training's multi_logloss: 0.0527708	valid_1's multi_logloss: 0.0936868
[1213]	training's multi_logloss: 0.0527147	valid_1's multi_logloss: 0.0936805
[1214]	training's multi_logloss: 0.0526608	valid_1's multi_logloss: 0.0936713
[1215]	training's multi_logloss: 0.0526127	valid_1's multi_logloss: 0.0936622
[1216]	training's multi_logloss: 0.0525582	valid_1's multi_logloss: 0.0936615
[1217]	training's multi_logloss: 0.0525104	valid_1's multi_logloss: 0.0936571
[1218]	training's multi_logloss: 0.052459	valid_1's multi_logloss: 0.09366
[1219]	training's multi_logloss: 0.0524024	valid_1's multi_logloss: 0.0936561
[1220]	training's multi_logloss: 0.0523507	valid_1's multi_logloss: 0.

In [14]:
gbm.best_iteration

262

In [179]:
y_pred=gbm.predict(test_x_all[modelCols])
pred_label=np.argmax(y_pred,axis=1)
macro_f1=f1_score(test_y,pred_label,average='macro')
print(macro_f1)

0.8887017921235776


In [83]:
y_pred=gbm.predict(test_x_all[modelCols])
pred_label=np.argmax(y_pred,axis=1)
macro_f1=f1_score(test_y,pred_label,average='macro')
print(macro_f1)

0.8824137542879823


In [146]:
pd.crosstab(test_y,pred_label)

col_0,0,1,2,3,4
label_index,,,,,
0,2095,3,360,335,0
1,5,1978,39,118,33
2,95,113,70116,310,103
3,107,52,155,4085,5
4,1,5,60,6,511


In [182]:
pd.crosstab(test_y,pred_label)

col_0,0,1,2,3,4
label_index,,,,,
0,2094,3,359,336,1
1,4,1976,41,119,33
2,86,112,70132,309,98
3,96,48,155,4098,7
4,0,3,63,8,509


In [181]:
submit_pred=gbm.predict(submitData[modelCols])
submit_pred_label=np.argmax(submit_pred,axis=1)
submitData["pred_label"]=lb.inverse_transform(submit_pred_label)
submitData.groupby("pred_label")["user"].count()

D:\Users\fjc-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


pred_label
0F2E4CC10EDBE80F     1175
56AFA2A526F96CC9     2353
7C26FADD409BD4B9    70387
816A9BEBED2D7C99     4745
C7E2941B65C6CCD6      747
Name: user, dtype: int64

In [183]:
submitData[["user","pred_label"]].to_csv("E:\zte\data\chongqin_lgbm_10.csv",index=False,header=False)

In [180]:
for x,y in sorted(zip(modelCols,gbm.feature_importance("gain")),key=lambda x:x[1]):
    print(x,y)

dt_m_1102_type 295.20498421788216
dt_m_1108_type 307.8350947275758
dt_m_1087_pref 426.760252058506
dt_m_1086_type 436.3055199831724
dt_m_1617_type 451.9303575158119
app5_visits_type 480.97737285494804
app6_visits_type 778.1600888669491
dt_m_1633_type 870.1060969755054
app4_visits_type 877.5506430268288
app7_visits_type 938.691976800561
last_year_capture_user_flag 1007.830913208425
app3_visits_type 1094.1372305899858
dt_m_1630_type 1416.8544686585665
app_pref 1448.9579094350338
gender 1534.6366300359368
brand 1715.1186976283789
app3_visits 2020.9276057109237
credit_level 2319.110273256898
dt_m_1633 2348.9576244167984
app2_visits_type 2433.723778195679
dt_m_1108 2539.5597406476736
zjctdur_perc 2543.583195760846
inet_pd_inst_cnt 2694.2075091451406
bjctdur_per_times 2923.6657354682684
dt_m_1087_type 2951.4451243281364
prdOpen_net_dur 3064.532713893801
message_send_perc 3366.987175360322
app7_visits 3550.34235150367
dt_m_1594_type 3597.330715432763
market_price_per_mon 3670.864648432471
dt_